In [1]:
import pandas as pd
import torchtext
from torchtext.vocab import Vectors
from torchtext.legacy import data
import torch
from torch import nn
import torch.nn.functional as F
from d2l import torch as d2l
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from sklearn import metrics
import sys
sys.path.append("D:/Experiment")
from tqdm import tqdm
from MyKu import training
from MyKu import processing
from torchtext.vocab import Vectors
from spacy.lang.en import English

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)


cuda:0


In [2]:
processing.create_OLID()
# processing.set_olid_train_data(processing.ORIGIN_DATASET_PATH + '/OLID')
# processing.set_olid_testA_data(processing.ORIGIN_DATASET_PATH + '/OLID')


In [2]:
def tokenizer(text):  # create a tokenizer function
    """
    定义分词操作
    """
    return processing.Pre_processing_tweets().tokenize_process(text)


def DataLoader():

    TEXT = data.Field(sequential=True, tokenize=tokenizer,
                      lower=True, include_lengths=True, fix_length=20)
    LABEL = data.Field(sequential=False, use_vocab=False)
    # 假如train.csv文件并不是只有两列，比如1、3列是review和polarity，2列是我们不需要的数据，
    # 那么就要添加一个全是None的元组， fields列表存储的Field的顺序必须和csv文件中每一列的顺序对应，

    train_fields = [(None, None), ('label', LABEL),  ('tweet', TEXT)]
    # train_fields = [(None, None), (None, None), (None, None), (None, None), ('text', TEXT), ('task1', LABEL)]
    train_data = data.TabularDataset(
        path='D:/Experiment/datasets/SEM2018/train.tsv',
        # path='D:/Experiment/datasets/EXIST2021/train.tsv',
        format='tsv',
        fields=train_fields,
        skip_header=True  # 是否跳过文件的第一行
    )
    test_fields = [(None, None), ('label', LABEL),  ('tweet', TEXT)]
    # test_fields = [(None, None), (None, None), (None, None), (None, None), ('text', TEXT), ('task1', LABEL)]
    test_data = data.TabularDataset(
        path='D:/Experiment/datasets/SEM2018/test.tsv',
        # path='D:/Experiment/datasets/EXIST2021/test.tsv',
        format='tsv',
        fields=test_fields,
        skip_header=True  # 是否跳过文件的第一行
    )
    return train_data, test_data, TEXT, LABEL


In [3]:
train_data, test_data, TEXT, LABEL = DataLoader()


vectors = Vectors(name='glove.6B.300d.txt', cache=processing.EMBEDDING_PATH)

TEXT.build_vocab(train_data,  # 建词表是用训练集建，不要用验证集和测试集
                  max_size=400000, # 单词表容量
                  vectors=vectors, # 还有'glove.840B.300d'已经很多可以选
                  unk_init=torch.Tensor.normal_ # 初始化train_data中不存在预训练词向量词表中的单词
)

In [4]:
from operator import truediv

# BiLSTM + Attention 
class BiRNN(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, n_class, bidirectional, dropout, **kwargs):
        super(BiRNN, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.encoder = nn.LSTM(
            embed_size, num_hiddens, num_layers=num_layers, bidirectional=True, dropout=dropout, batch_first=True)
        self.n_class = n_class  # 类别数
        self.bidirectional = bidirectional  # 控制是否双向LSTM
        if self.bidirectional:
            self.decoder1 = nn.Linear(num_hiddens * 2, num_hiddens)
            self.decoder2 = nn.Linear(num_hiddens, n_class)
        else:
            self.decoder1 = nn.Linear(num_hiddens * 2, num_hiddens)
            self.decoder2 = nn.Linear(num_hiddens, n_class)
        self.weight_W = nn.Parameter(torch.Tensor(2 * num_hiddens, 2 * num_hiddens))
        self.weight_proj = nn.Parameter(torch.Tensor(2 * num_hiddens, 1))
        self.t = nn.Parameter(torch.Tensor(num_hiddens, 2))
        nn.init.uniform_(self.weight_W, -0.1, 0.1)
        nn.init.uniform_(self.weight_proj, -0.1, 0.1)


    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        # print(embeddings.shape)
        self.encoder.flatten_parameters()
        outputs, _ = self.encoder(embeddings.permute(1, 0, 2))
        # print(outputs.shape)
        u = torch.tanh(torch.matmul(outputs, self.weight_W))
        # print(u.shape)
        att = torch.matmul(u, self.weight_proj)
        att_score = F.softmax(att, dim=1)
        # print(att_score.shape)
        score_x = outputs * att_score
        encoding = torch.sum(score_x, dim=1)
        # encoding = torch.cat((outputs[0], outputs[-1]), dim=1)
        outs = self.decoder1(encoding)
        outs = self.decoder2(outs)
        # print(outs.shape)
        return outs


In [24]:
# X = torch.ones((4, 6), dtype=torch.long).to(device=DEVICE)
model = BiRNN(len(TEXT.vocab), 300, 200, 1, 2, True, 0.7)
model.to(DEVICE)

d:\Software\anaconda3\envs\pytorch_env\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.7 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


BiRNN(
  (embedding): Embedding(9199, 300)
  (encoder): LSTM(300, 200, batch_first=True, dropout=0.7, bidirectional=True)
  (decoder1): Linear(in_features=400, out_features=200, bias=True)
  (decoder2): Linear(in_features=200, out_features=2, bias=True)
)

In [25]:
pretrained_embeddings = TEXT.vocab.vectors
# print(pretrained_embeddings.shape)
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(300)
model.embedding.weight.data[PAD_IDX] = torch.zeros(300)

In [7]:
train_iter, test_iter = data.BucketIterator.splits(
    (train_data, test_data), batch_size=64, sort_within_batch=True, sort_key=lambda x : len(x.tweet), device=DEVICE
)

In [8]:
def train(model, train_iter, optimizer, loss, epoch):
    model.train()
    epoch_loss = 0
    num_sample = 0
    correct = 0
    for batch in tqdm(train_iter, desc=f"Training Epoch {epoch}", colour='red'):
        optimizer.zero_grad()
        text, text_len = batch.tweet
        label = batch.label
        # text, text_len = batch.text
        # label = batch.task1
        output = model(text)
        pred_y = torch.argmax(output, dim=1)
        correct += torch.sum(pred_y == label)
        l = loss(output, label)
        l.backward()
        epoch_loss += l.item()
        num_sample += len(batch)
        optimizer.step()
    print(
        f'\tTrain Loss: {epoch_loss / num_sample:.3f} | Train Acc: {correct.float() / num_sample* 100:.2f}%')


def test(model, test_iter):
    true_y, pred_y = [], []
    for batch in tqdm(test_iter, desc=f"Testing", colour='green'):
        text, text_len = batch.tweet
        label = batch.label
        # text, text_len = batch.text
        # label = batch.task1
        with torch.no_grad():
            output = model(text)
            pred_y.extend(output.argmax(dim=1).tolist())
            true_y.extend(label.tolist())
    print(metrics.confusion_matrix(true_y, pred_y))
    print(metrics.classification_report(true_y, pred_y))
    print(f'Acc : {metrics.accuracy_score(true_y, pred_y)}\t F1: {metrics.f1_score(true_y, pred_y, average="macro")}')


In [26]:
lr, num_epochs = 0.0001, 20
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss = nn.CrossEntropyLoss()
for epoch in range(1, num_epochs+1):
    train(model, train_iter, optimizer, loss, epoch)
    test(model, test_iter)

Training Epoch 1: 100%|██████████| 60/60 [00:00<00:00, 107.12it/s]


	Train Loss: 0.011 | Train Acc: 51.30%


Testing: 100%|██████████| 13/13 [00:00<00:00, 282.35it/s]


[[ 27 446]
 [  7 304]]
              precision    recall  f1-score   support

           0       0.79      0.06      0.11       473
           1       0.41      0.98      0.57       311

    accuracy                           0.42       784
   macro avg       0.60      0.52      0.34       784
weighted avg       0.64      0.42      0.29       784

Acc : 0.4221938775510204	 F1: 0.33977658678593936


Training Epoch 2: 100%|██████████| 60/60 [00:00<00:00, 104.34it/s]


...

[[323 150]
 [120 191]]
              precision    recall  f1-score   support

           0       0.73      0.68      0.71       473
           1       0.56      0.61      0.59       311

    accuracy                           0.66       784
   macro avg       0.64      0.65      0.65       784
weighted avg       0.66      0.66      0.66       784

Acc : 0.6556122448979592	 F1: 0.6455648726123182


Training Epoch 20: 100%|██████████| 60/60 [00:00<00:00, 100.43it/s]


	Train Loss: 0.001 | Train Acc: 98.72%


Testing: 100%|██████████| 13/13 [00:00<00:00, 382.00it/s]

[[318 155]
 [114 197]]
              precision    recall  f1-score   support

           0       0.74      0.67      0.70       473
           1       0.56      0.63      0.59       311

    accuracy                           0.66       784
   macro avg       0.65      0.65      0.65       784
weighted avg       0.67      0.66      0.66       784

Acc : 0.6568877551020408	 F1: 0.6485154537803222
